In [1]:
!pip install Flask


In [2]:
from flask import Flask, render_template, request
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
data = pd.read_excel('hospital_data.xlsx')

# Extract features and target variable
X = data[['AGE', 'GENDER', 'MARITAL STATUS', 'KEY COMPLAINTS -CODE', 'BODY WEIGHT', 'BODY HEIGHT', 'HR PULSE', 'RR']]
y = data['TOTAL COST TO HOSPITAL']

# Handle categorical variables
X = pd.get_dummies(X, columns=['GENDER', 'MARITAL STATUS'])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Predict on test set
y_pred = model.predict(X_test_scaled)

# Calculate SSR, MSE, and R2
SSR = ((y_pred - y_test) ** 2).sum()
MSE = mean_squared_error(y_test, y_pred)
R2 = r2_score(y_test, y_pred)

# Flask app
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        # Collect user inputs from the form
        age = float(request.form['age'])
        gender = float(request.form['gender'])
        marital_status = float(request.form['marital_status'])
        key_complaints_code = float(request.form['key_complaints_code'])
        body_weight = float(request.form['body_weight'])
        body_height = float(request.form['body_height'])
        hr_pulse = float(request.form['hr_pulse'])
        rr = float(request.form['rr'])

        # Preprocess user inputs
        user_input = [[age, gender, marital_status, key_complaints_code, body_weight, body_height, hr_pulse, rr]]
        user_input_scaled = scaler.transform(user_input)

        # Make prediction
        prediction = model.predict(user_input_scaled)

        return render_template('result.html', prediction=prediction[0], SSR=SSR, MSE=MSE, R2=R2)

if __name__ == '__main__':
    app.run(debug=True)


KeyError: "None of [Index(['AGE', 'GENDER', 'MARITAL STATUS', 'KEY COMPLAINTS -CODE',\n       'BODY WEIGHT', 'BODY HEIGHT', 'HR PULSE', 'RR'],\n      dtype='object')] are in the [columns]"

<h1>index.html</h1>


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Medical Expenses Prediction</title>
</head>
<body>
    <h1>Medical Expenses Prediction</h1>
    <form action="/predict" method="post">
        <label for="age">Age:</label>
        <input type="text" id="age" name="age"><br><br>

        <label for="gender">Gender:</label>
        <select id="gender" name="gender">
            <option value="0">Male</option>
            <option value="1">Female</option>
        </select><br><br>

        <label for="marital_status">Marital Status:</label>
        <select id="marital_status" name="marital_status">
            <option value="0">Single</option>
            <option value="1">Married</option>
        </select><br><br>

        <label for="key_complaints_code">Key Complaints Code:</label>
        <input type="text" id="key_complaints_code" name="key_complaints_code"><br><br>

        <label for="body_weight">Body Weight:</label>
        <input type="text" id="body_weight" name="body_weight"><br><br>

        <label for="body_height">Body Height:</label>
        <input type="text" id="body_height" name="body_height"><br><br>

        <label for="hr_pulse">HR Pulse:</label>
        <input type="text" id="hr_pulse" name="hr_pulse"><br><br>

        <label for="rr">RR:</label>
        <input type="text" id="rr" name="rr"><br><br>

        <input type="submit" value="Predict">
    </form>
</body>
</html>


<h1> result.html </h2>

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Result</title>
</head>
<body>
    <h1>Medical Expenses Prediction Result</h1>
    <p>Predicted Total Cost to Hospital: {{ prediction }}</p>
    <p>Sum Squared Regression (SSR): {{ SSR }}</p>
    <p>Mean Squared Error (MSE): {{ MSE }}</p>
    <p>R-squared (R2) Score: {{ R2 }}</p>
</body>
</html>
